In [77]:
import numpy as np
from scipy.stats import norm


n = 30  # sample size
x = np.arange(1, n+1) / 10  # fixed x values = 1/10 ~ n/10
beta0 = 3   # true intercept
alpha = 0.05  # significance level
z = norm.ppf(1 - alpha/2)  # z_{1 - alpha/2}
B = int(1e4)  # number of experiments

**1-(a).**

In [78]:
np.random.seed(0)  # for reproducibility
intervals = np.zeros((B, 2))    # to store the confidence intervals (each row is an interval)
for b in range(B):
    y = beta0 + np.random.randn(n)  # generate y ~ N(3, 1) = N(beta0, 1)
    y_bar = np.mean(y)  # sample mean
    intervals[b, 0] = y_bar - z / np.sqrt(n)  # lower bound
    intervals[b, 1] = y_bar + z / np.sqrt(n)  # upper bound

# check how many intervals contain the true value of beta0
is_covered = (intervals[:, 0] <= beta0) & (beta0 <= intervals[:, 1])
print(f'Coverage: {np.mean(is_covered):.4f} (expected: {1 - alpha})')

Coverage: 0.9530 (expected: 0.95)


**1-(b).**

In [79]:
np.random.seed(1)  # for reproducibility
intervals = np.zeros((B, 2))    # to store the confidence intervals (each row is an interval)
for b in range(B):
    y = beta0 + np.random.randn(n)  # generate y ~ N(3, 1) = N(beta0, 1)
    
    # ols 
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    beta_hat1 = np.sum((x - x_bar) * (y - y_bar)) / np.sum((x - x_bar)**2)
    beta_hat0 = y_bar - beta_hat1 * x_bar

    sigma = np.sqrt(1 / n + x_bar**2 / np.sum((x - x_bar)**2)) # std of beta_hat0
    intervals[b, 0] = beta_hat0 - z * sigma
    intervals[b, 1] = beta_hat0 + z * sigma

# check how many intervals contain the true value of beta0
is_covered = (intervals[:, 0] <= beta0) & (beta0 <= intervals[:, 1])
print(f'Coverage: {np.mean(is_covered):.4f} (expected: {1 - alpha})')

Coverage: 0.9506 (expected: 0.95)


**1-(c).**

In [80]:
np.random.seed(3)  # for reproducibility
intervals = np.zeros((B, 2))    # to store the confidence intervals (each row is an interval)
is_linear_model = np.zeros(B)  # 1 if the linear model is chosen (|beta_hat1| > threshold) and 0 otherwise

for b in range(B):
    y = beta0 + np.random.randn(n)  # generate y ~ N(3, 1) = N(beta0, 1)
    
    # ols 
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    beta_hat1 = np.sum((x - x_bar) * (y - y_bar)) / np.sum((x - x_bar)**2)
    beta_hat0 = y_bar - beta_hat1 * x_bar

    if np.abs(beta_hat1) <= z / np.sqrt(np.sum((x - x_bar)**2)):
        intervals[b, 0] = y_bar - z / np.sqrt(n)
        intervals[b, 1] = y_bar + z / np.sqrt(n)

    else:
        is_linear_model[b] = 1
        sigma = np.sqrt(1 / n + x_bar**2 / np.sum((x - x_bar)**2)) # std of beta_hat0
        intervals[b, 0] = beta_hat0 - z * sigma
        intervals[b, 1] = beta_hat0 + z * sigma

# check how many intervals contain the true value of beta0
is_covered = (intervals[:, 0] <= beta0) & (beta0 <= intervals[:, 1])
print(f'Coverage: {np.mean(is_covered):.4f} (expected: {1 - alpha})')
print(f'Linear model valid: {np.mean(is_linear_model):.3f}')
print(f'Conditional coverage when linear model is chosen: {np.mean(is_covered[is_linear_model == 1]):.4f} (expected: {1 - alpha})')
print(f'Conditional coverage when linear model is rejected: {np.mean(is_covered[is_linear_model == 0]):.4f} (expected: {1 - alpha})')

Coverage: 0.9244 (expected: 0.95)
Linear model valid: 0.050
Conditional coverage when linear model is chosen: 0.4405 (expected: 0.95)
Conditional coverage when linear model is rejected: 0.9501 (expected: 0.95)


**1-(d).**

In [81]:
np.random.seed(4)  # for reproducibility
intervals = np.zeros((B, 2))    # to store the confidence intervals (each row is an interval)
is_linear_model = np.zeros(B)  # 1 if the linear model is chosen (|beta_hat1| > threshold) and 0 otherwise

for b in range(B):
    y = beta0 + np.random.randn(n)  # generate y ~ N(3, 1) = N(beta0, 1)
    
    # data split (validation data = test the linear model (1 / 3), remaining data = construct the linear model (2 / 3))
    val_id = np.random.choice(n, n // 3, replace=False)
    rest_id = np.setdiff1d(np.arange(n), val_id)
    x_val = x[val_id]
    y_val = y[val_id]
    x_rest = x[rest_id]
    y_rest = y[rest_id]
    n_rest = len(y_rest)

    # ols with the validation data
    x_bar = np.mean(x_val)
    y_bar = np.mean(y_val)
    beta_hat1 = np.sum((x_val - x_bar) * (y_val - y_bar)) / np.sum((x_val - x_bar)**2)
    
    if np.abs(beta_hat1) <= z / np.sqrt(np.sum((x_val - x_bar)**2)):
        y_bar = np.mean(y_rest)
        intervals[b, 0] = y_bar - z / np.sqrt(n_rest)
        intervals[b, 1] = y_bar + z / np.sqrt(n_rest)

    else:
        is_linear_model[b] = 1

        # ols with the remaining data
        x_bar = np.mean(x_rest)
        y_bar = np.mean(y_rest)
        beta_hat1 = np.sum((x_rest - x_bar) * (y_rest - y_bar)) / np.sum((x_rest - x_bar)**2)
        beta_hat0 = y_bar - beta_hat1 * x_bar
        sigma = np.sqrt(1 / n_rest + x_bar**2 / np.sum((x_rest - x_bar)**2))
        intervals[b, 0] = beta_hat0 - z * sigma
        intervals[b, 1] = beta_hat0 + z * sigma

# check how many intervals contain the true value of beta0
is_covered = (intervals[:, 0] <= beta0) & (beta0 <= intervals[:, 1])
print(f'Coverage: {np.mean(is_covered):.4f} (expected: {1 - alpha})')
print(f'Linear model valid: {np.mean(is_linear_model):.3f}')
print(f'Conditional coverage when linear model is chosen: {np.mean(is_covered[is_linear_model == 1]):.4f} (expected: {1 - alpha})')
print(f'Conditional coverage when linear model is rejected: {np.mean(is_covered[is_linear_model == 0]):.4f} (expected: {1 - alpha})')

Coverage: 0.9517 (expected: 0.95)
Linear model valid: 0.052
Conditional coverage when linear model is chosen: 0.9540 (expected: 0.95)
Conditional coverage when linear model is rejected: 0.9516 (expected: 0.95)


**Discussion**

We have $\hat{\beta}_1 \neq 0$ with probability $0.05$, which is the type I error of testing the significance of the true slope $\beta_1 = 0$. When this happens, i.e., when we commit this type I error, the estimated intercept term $\hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x}$ differs much from the mean estimator $\bar{y}$; accordingly, the interval based on $\hat{\beta}_0$ will not have the desired coverage probability of $0.95$.

See "conditional coverage when linear model is chosen" in (c); the conditional coverage is significantly lower than $0.95$.

On the other hand, in (d), even when we commit the type I error of testing the significance of $\beta_1$ using the validation data, we construct the interval based on the OLS result of the remaining data independent of the validation data. In this case, the conditional coverage has to be $0.95$ even if the linear model is chosen as verified in (b).